In [1]:
%cd ..

/notebooks/v2


In [2]:
!python -u run.py 'data' 'predictions.npz'

['run.py', 'data', 'predictions.npz']
run.py--data--predictions.npz
Global seed set to 0
data
predictions.npz
['tr.csv', 'TEST.feather', 'XT.pt', '5000.json', 'click_categories.csv', '10000.json', 'sample_submission.csv', 'clickstreams_events.feather', 'bank.feather', 'transactions.feather', 'tr.feather', 'rtk.feather', '10000.feather', '5000.feather', 'currency_rk.csv', 'transactions_events_uids.feather', 'cl.csv', 'mcc_codes.csv', '19623.feather', '15000.feather', 'TRAIN.feather', 'transactions_events.feather', 'train_matching.csv', 'clickstreams_events_uids.feather', '15000.json', 'cl.feather', 'XC.pt', 'YC.pt', 'puzzle.csv', 'YT.pt', '19623.json']
14797017
100%|█████████████████████████████| 2611403/2611403 [00:07<00:00, 328447.20it/s]
Sequential2Splitter __init__...
Sequential2Splitter run...
Sequential2Splitter get_tensors...
100%|██████████████████████████████████████| 2463/2463 [00:14<00:00, 164.37it/s]
Sequential2Splitter pad...
100%|██████████████████████████████████| 16636/1

In [3]:
1/0

ZeroDivisionError: division by zero

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from lib.base import *
from lib.data.splitter.sequential2 import Sequential2Splitter
from lib.run.args import Args
from lib.data.datamodule.test2 import Test2
from lib.run.model import Model

In [ ]:
import sys
import pickle
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import gc


# data, output_path = sys.argv[1:]
data = 'data'
output_path = 'predictions.npz'


transactions = pd.read_csv(
    f'{data}/tr.csv')
bankclient_embed = transactions.pivot_table(index = 'user_id', 
                        values=['transaction_amt'],
                        columns=['mcc_code'],
                        aggfunc=['sum','mean', 'count']).fillna(0)
bankclient_embed.columns = [f'{str(i[0])}-{str(i[2])}' for i in bankclient_embed.columns]

del transactions
gc.collect()

clickstream = pd.read_csv(
    f'{data}/cl.csv')
clickstream_embed = clickstream.pivot_table(index = 'user_id', 
                        values=['timestamp'],
                        columns=['cat_id'],
                        aggfunc=['count']).fillna(0)
clickstream_embed.columns = [f'{str(i[0])}-{str(i[2])}' for i in clickstream_embed.columns]
clickstream_embed.loc[0] = np.empty(len(clickstream_embed.columns))

del clickstream
gc.collect()

dtype_clickstream = list()
for x in clickstream_embed.dtypes.tolist():
    if x=='int64':
        dtype_clickstream.append('int16')
    elif(x=='float64'):
        dtype_clickstream.append('float32')
    else:
        dtype_clickstream.append('object')

dtype_clickstream = dict(zip(clickstream_embed.columns.tolist(),dtype_clickstream))
clickstream_embed = clickstream_embed.astype(dtype_clickstream)

dtype_bankclient = list()
for x in bankclient_embed.dtypes.tolist():
    if x=='int64':
        dtype_bankclient.append('int16')
    elif(x=='float64'):
        dtype_bankclient.append('float32')
    else:
        dtype_bankclient.append('object')

dtype_bankclient = dict(zip(bankclient_embed.columns.tolist(),dtype_bankclient))
bankclient_embed = bankclient_embed.astype(dtype_bankclient)

list_of_rtk = list(clickstream_embed.index.unique())
list_of_bank= list(bankclient_embed.index.unique())

submission = pd.DataFrame(list_of_bank, columns=['bank'])
submission['rtk'] = submission['bank'].apply(lambda x: list_of_rtk)

with open("full_list_of_features", "rb") as fp:   # Unpickling
    full_list_of_features = pickle.load(fp)

model = CatBoostClassifier()
model.load_model('model.cbm',  format='cbm')

submission_ready = []

batch_size = 200
# batch_size = 20*2
num_of_batches = int((len(list_of_bank))/batch_size)+1

for i in tqdm(range(num_of_batches)):
    bank_ids = list_of_bank[(i*batch_size):((i+1)*batch_size)]
    if len(bank_ids) != 0:
        part_of_submit = submission[submission['bank'].isin(bank_ids)].explode('rtk')
        part_of_submit = part_of_submit.merge(bankclient_embed, how='left', left_on='bank', right_index=True
                                    ).merge(clickstream_embed, how='left', left_on='rtk', right_index=True).fillna(0)

        for i in full_list_of_features:
            if i not in part_of_submit.columns:
                part_of_submit[i] = 0


        part_of_submit['predicts'] = model.predict_proba(
            part_of_submit[full_list_of_features],
#             task_type='GPU'
        )[:,1]
        part_of_submit = part_of_submit[['bank', 'rtk', 'predicts']]

        zeros_part = pd.DataFrame(bank_ids, columns=['bank'])
        zeros_part['rtk'] = 0.
        zeros_part['predicts'] = 3.8

        part_of_submit = pd.concat((part_of_submit, zeros_part))

        part_of_submit = part_of_submit.sort_values(by=['bank', 'predicts'], ascending=False).reset_index(drop=True)
        part_of_submit = part_of_submit.pivot_table(index='bank', values='rtk', aggfunc=list)
        part_of_submit['rtk'] = part_of_submit['rtk'].apply(lambda x: x[:100])
        part_of_submit['bank'] = part_of_submit.index
        part_of_submit = part_of_submit[['bank', 'rtk']]
        submission_ready.extend(part_of_submit.values)

submission_final = np.array(submission_ready, dtype=object)

print(submission_final.shape)
print(submission_final)
np.savez(output_path, submission_final)


In [ ]:
nn=np.load('npz/nn.npz',allow_pickle=True)['arr_0']
len(nn[0][1])

In [ ]:
cb=np.load('npz/cb.npz',allow_pickle=True)['arr_0']
len(cb[0][1])

In [ ]:
1/0

In [ ]:
1/5

In [ ]:
k = 2930
tr = io.load('data/transactions.feather')
tr = tr[tr.user_id.isin(
    tr.user_id.unique()[:k])]
assert tr.user_id.nunique()==k
io.save(tr,'data/tr.csv')

In [ ]:
k = 2463
cl = io.load('data/5000.feather')
cl = cl[cl.user_id.isin(
    cl.user_id.unique()[:k])]
assert cl.user_id.nunique()==k
io.save(cl,'data/cl.csv')

In [ ]:
# data_dir, pred_file =sys.argv[1].split('--')[1:]
# data_dir = Path(data_dir)
# pred_file = Path(pred_file)
# print(data_dir) #/data
# print(pred_file) #/output/predictions.npz
# print(os.listdir(data_dir)) #['clickstream.csv', 'transactions.csv']

data_dir = Path('data')
pred_file = Path('predictions.npz')

root = Path('json')
event_encoder = Encoder(root,'event')
event_encoder.load()
uid_encoder = Encoder(root,'uid') 
uid_encoder.load()
uid = 'user_id'
ts = 'timestamp'

#cl = pd.read_csv(data_dir/'clickstream.csv')
cl = pd.read_csv(data_dir/'cl.csv')
# print(len(cl))
# if len(cl)>1000:
#     1/0   

cl[ts] = cl[ts].progress_apply(pd.Timestamp)
event = 'cat_id'
cl[event] = cl[event].apply(
    lambda x: f'rtk_{x}')
cl[event]=event_encoder.transform(cl[event])
cl[uid] = uid_encoder.transform(cl[uid])

# tr = pd.read_csv(
#     data_dir/'transactions.csv')
tr = pd.read_csv(data_dir/'tr.csv')
tr = tr.rename(
    columns={'transaction_dttm': ts})
tr[ts] = tr[ts].progress_apply(pd.Timestamp)
event = 'mcc_code'
tr[event] = tr[event].apply(
    lambda x: f'bank_{x}')
tr[event]=event_encoder.transform(tr[event])
tr[uid] = uid_encoder.transform(tr[uid]) 

bank = sorted(
    tr.user_id.unique().tolist())
rtk = sorted(
    cl.user_id.unique().tolist())   
max_len = max(len(bank),len(rtk))
df = pd.DataFrame(index=range(max_len))

splitter = Sequential2Splitter()
XC,YC,XT,YT = splitter.run(cl,tr)
#       df['bank'] = (bank+[-1]*len(bank))[:max_len]
#       df['rtk'] = (rtk+[-1]*len(rtk))[:max_len]

df['bank'] = (bank+bank)[:max_len]
df['rtk'] = (rtk+rtk)[:max_len]

df = df.fillna(-1)

del cl, tr
gc.collect()

a = Args(
    splitter = 'Sequential',
    splitter_pp = dict(
        n_days_in_sample = 30,
        bank_quantile = 0.9,
        rtk_quantile = 0.9,
    ),    
    n_folds = 3,# 1000 == 'full train'
    fold = 0,

    fit_limit = 1.,
    val_limit = 1.,

    batch_size = 32,    
    lr = 2e-3,
    n_epochs = 10,
    check_val_every_n_epoch = 1,

    bb_pp = dict(
        block_size = 16,
        hidden_size = 128,
        intermediate_size = 128,
        num_attention_heads = 1,
        num_hidden_layers = 1,
        num_random_blocks = 1,
    ),

    loss = 'MarginLoss',
    loss_pp = dict(),

    use_unmatched = bool(0),

    miner = None,
    miner_pp = dict(),

    avg_loss = 'mean',
    avg_pred = 'mean',
)
a.update()
a.bank_len = XT.shape[1]
a.rtk_len = XC.shape[1] 

def collate(DD):
    AB = [A+B for A in 'XY' for B in 'TC']
    kk = AB+['MT','MC']+['bank','rtk','M']
    B = {k:[] for k in kk}
    for D in DD:
        for k in B:
            if k in D:
                B[k].append(D[k])
    for k in B:
        if k in AB+['MT','MC']:
            B[k] = torch.cat(B[k])
        if k in ['bank','rtk','M']:
            B[k] = torch.tensor(B[k])
    return B

c = Args()
c.event_encoder = event_encoder
c.uid_encoder = uid_encoder
d = Args()
d.P = df
d.XT = XT
d.XC = XC
d.YT = YT
d.YC = YC
c.test = Test2(d, collate, a)

model = Model(a,c)

callbacks = [
    pl.callbacks.model_checkpoint.ModelCheckpoint(
        save_weights_only = bool(0),
        filename = '{R1} {MRR} {P}', 
        monitor = 'R1', 
        verbose = False,
        save_last = bool(1),
        save_top_k = 1, 
        mode = 'max', 
    ),
]
trainer = pl.Trainer(
    accumulate_grad_batches = a.acc_batches,
#            val_check_interval=a.val_check_interval,
    check_val_every_n_epoch=a.check_val_every_n_epoch,
    num_sanity_val_steps = 0,
    deterministic = bool(0) if a.avg_loss=='median' or a.avg_pred=='median' else bool(1),
    benchmark = bool(1),
    gpus = a.gpus,
    precision = a.precision,
    logger = pl.loggers.CSVLogger(
        str(a.log_dir), name=a.exp_name),
    callbacks = callbacks,
    max_epochs = a.n_epochs,
    limit_train_batches = a.fit_limit,
    limit_val_batches = a.val_limit,
)   

p = a.log_dir
p /= a.exp_name
p /= 'version_0'
p /= 'checkpoints'
a.ckpt = p/'last.ckpt'
print(a.ckpt.stem)
BB = trainer.predict(
    model, c.test, ckpt_path=a.ckpt)  
pred = model.predict_epoch_end(BB)
#         print(pred.sample().T)

path = a.csv_dir/f'{a.ckpt.stem}.csv'
c = 'rtk_list'
#         pred[c] = pred[c]\
#             .apply(lambda x: str(x))\
#             .replace("'", '', regex=True)
pred[c] = pred[c].apply(
    lambda x: ([0.0, 0]+x)[:100])
print(pred.values)
np.savez(str(pred_file), pred.values)


In [ ]:
os.rename(
    'predictions.npz',
    'npz/nn.npz'
)

In [ ]:
x = np.load(
    'predictions.npz', allow_pickle=bool(1))

In [ ]:
x['arr_0']

In [ ]:
pip install catboost

In [ ]:
1

In [ ]:
tr.columns

In [ ]:
cl.columns

In [ ]:
 len((rtk+rtk+rtk+rtk+rtk+rtk)[:max_len])

In [ ]:
 len((bank+bank+bank+bank+bank+bank+bank)[:max_len])